# Question 1. dlt Version

In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [6]:
!dlt --version

dlt 1.12.3


## dlt Resourse

In [2]:
import dlt
import requests
import pandas as pd
from datetime import datetime

In [9]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

# Question 2. dlt pipeline

In [10]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [12]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.70it/s]


Run started at 2025-07-05 15:17:54.845522+00:00 and COMPLETED in 7.01 seconds with 4 steps.
Step extract COMPLETED in 0.48 seconds.

Load package 1751728677.2411158 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.09 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1751728677.2411158 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 4.05 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.04 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/03-dlt/db.qdrant location to store data
Load package 1751728677.2411158 is LOADED and contains no failed jobs

Step run COMPLETED in 7.01 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.04 seconds
1 load package(s) were loaded to destination

**How many rows were inserted into the zoomcamp_data collection?** 948 row(s)

# Question 3. Embeddings